<a href="https://colab.research.google.com/github/shubham3032002/Airline_customer_statisfaction_prediction_with_MLOPS/blob/main/text_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade transformers[sentencepiece] datasets sacrebleu rouge_score py7zr --no-cache-dir

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.2 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 3.1.0
    Uninstalling datasets-3.1.0:
      Successfully uninstalled datasets-3.1.0


In [ ]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.47.0
Uninstalling transformers-4.47.0:
  Successfully uninstalled transformers-4.47.0
Found existing installation: accelerate 1.2.0
Uninstalling accelerate-1.2.0:
  Successfully uninstalled accelerate-1.2.0
  Using cached transformers-4.47.0-py3-none-any.whl.metadata (43 kB)
  Using cached accelerate-1.2.0-py3-none-any.whl.metadata (19 kB)
Using cached transformers-4.47.0-py3-none-any.whl (10.1 MB)
Using cached accelerate-1.2.0-py3-none-any.whl (336 kB)


In [ ]:
!pip install datasets

In [ ]:
!pip install evaluate

In [ ]:
from transformers import pipeline, set_seed
from datasets import load_dataset,load_from_disk
import matplotlib.pyplot as plt
import pandas as pd
import evaluate

from transformers import AutoModelForSeq2SeqLM,AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
device="cuda" if torch.cuda.is_available()  else "cpu"
device

'cuda'

In [ ]:
model_ckpt ="google/pegasus-cnn_dailymail"
tokenizer =AutoTokenizer.from_pretrained(model_ckpt)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
model_pepasus=AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
dataset_samsung=load_dataset("samsum")

In [ ]:
# # Convert each dataset in the DatasetDict to a pandas DataFrame
# train_df = dataset_samsung['train'].to_pandas()  # For the 'train' dataset
# test_df = dataset_samsung['test'].to_pandas()    # For the 'test' dataset (if available)
# validation_df = dataset_samsung['validation'].to_pandas()  # For the 'validation' dataset (if available)



In [ ]:

dataset_samsung

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [ ]:
dataset_samsung["train"]["dialogue"][1]

'Olivia: Who are you voting for in this election? \r\nOliver: Liberals as always.\r\nOlivia: Me too!!\r\nOliver: Great'

In [ ]:
dataset_samsung["train"]["summary"][1]

'Olivia and Olivier are voting for liberals in this election. '

In [ ]:
split_lenght=[len(dataset_samsung[split]) for split in dataset_samsung]

In [ ]:
print(f"split lenght: {split_lenght}")
print(f"Features :{dataset_samsung['train'].column_names}")

split lenght: [14732, 819, 818]
Features :['id', 'dialogue', 'summary']


In [ ]:
print(dataset_samsung['test'][1]["dialogue"])

print("\n Summary")

print(dataset_samsung['test'][1]["summary"])

Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

 Summary
Eric and Rob are going to watch a stand-up on youtube.


In [ ]:
def convert_example_to_feature(example_batch):
  input_encoding =tokenizer(example_batch['dialogue'],max_length =1024,truncation=True)

  with tokenizer.as_target_tokenizer():
    target_encoding =tokenizer(example_batch['summary'],max_length =128,truncation=True)

  return{
      'input_ids': input_encoding['input_ids'],
      'attention_mask': input_encoding['attention_mask'],
      'labels': target_encoding['input_ids']
  }


In [ ]:
dataset_samsung_pt = dataset_samsung.map(
    convert_example_to_feature,
    batched=True,
    remove_columns=dataset_samsung["train"].column_names  # Removes 'id', etc.
)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [ ]:
dataset_samsung_pt['train']['attention_mask'][1]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [ ]:
from transformers import DataCollatorForSeq2Seq

Seq2Seq_data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model_pepasus
)

In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
from transformers import TrainingArguments, Trainer



# Training arguments
training_args = TrainingArguments(
    output_dir="pegasus-samsun",
    num_train_epochs=1,
    warmup_steps=500,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=int(1e6),
    gradient_accumulation_steps=16,
    report_to="none",
    remove_unused_columns=False,
    fp16=True  # Enable mixed precision training
)




/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model_pepasus,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=Seq2Seq_data_collator,
    train_dataset=dataset_samsung_pt['train'],
    eval_dataset=dataset_samsung_pt['validation']
)


<ipython-input-22-7736f9db114b>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
500,1.653400,1.485791


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=920, training_loss=1.8402713070745054, metrics={'train_runtime': 2903.7548, 'train_samples_per_second': 5.073, 'train_steps_per_second': 0.317, 'total_flos': 5528248038285312.0, 'train_loss': 1.8402713070745054, 'epoch': 0.9991854466467553})

Features: ['id', 'dialogue', 'summary']


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
#evalution
def generate_batch_sized_chucks(list_of_elements,batch_size):
  for i in range(0,len(list_of_elements),batch_size):
    yield list_of_elements[i: i + batch_size]

In [ ]:
def calulate_metric_on_test_sets(dataset,metric,model,tokenizer,
                                 batch_size=16,device=device,
                                 column_text="article",
                                 column_summary ="highlights"):
  article_batches =list(generate_batch_sized_chucks(dataset[column_text],batch_size))
  target_batches =list(generate_batch_sized_chucks(dataset[column_summary],batch_size))


  for article_batches,target_batches in tqdm(
      zip(article_batches,target_batches),total=len(article_batches)
  ):

    input =tokenizer(article_batches,max_length=1024,truncation=True,
                     padding="max_length",return_tensors="pt")




    summaries = model.generate(
    input_ids=inputs["input_ids"].to(device),
    attention_mask=inputs["attention_mask"].to(device),
    length_penalty=0.8,
    num_beams=8,
    max_length=128
          )

# The parameter for length penalty ensures that the model does not generate sequences that are too long.

# Finally, we decode the generated texts,
# replace the token, and add the decoded texts with the references to the metric.

    decoded_summaries = [
                    tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True)
                   for s in summaries
                                    ]

    decoded_summaries = [d.replace("", "") for d in decoded_summaries]


    metric.add_batch(predictions=decoded_summaries,references=target_batches)

    score =metric.compute()
    return score




In [ ]:


rouge_name=["rouge1","rouge2","rougeL","rougeLsum"]
rouge_mertic =evaluate.load('rouge')

In [ ]:
score = calulate_metric_on_test_sets(
    dataset_samsung['test'][0:10],
    rouge_mertic,
    trainer.model,
    tokenizer,
    batch_size=2,
    column_text='dialogue',
    column_summary='summary'
)



# Update this to directly access the score since 'mid' and 'fmeasure' attributes don't exist
rouge_dict = dict((rn, score[rn]) for rn in rouge_names) # Just use score[rn]

# Create DataFrame to display the results
pd.DataFrame(rouge_dict, index=[f'pegasus'])

  0%|          | 0/5 [00:00<?, ?it/s]


NameError: name 'inputs' is not defined